In [1]:
import os
import sys
home_dir = "../../"
module_path = os.path.abspath(os.path.join(home_dir))
if module_path not in sys.path:
    sys.path.append(module_path)

import numpy as np
import pandas as pd 
from models.aa_common.data_loader import get_population_freq_SNVs

In [2]:
variants_df = get_population_freq_SNVs(home_dir)
variants_df = variants_df[variants_df["class"]!="Zero-population"]


Log: Loading data ...
(2865836, 14)
Index(['snp_id', 'chrom_acc_version', 'chrom_pos', 'ref_allele', 'alt_allele',
       'prot_acc_version', 'prot_pos', 'wt', 'mut', 'wt_population',
       'mut_poulation', 'wt_freq', 'mt_freq', 'class'],
      dtype='object')
Zero-population    2042590
Singleton           462444
Ultra-rare          314013
Rare                 28622
Common               18167
Name: class, dtype: int64


In [3]:
# merging dbNSFP extracted scores with popu-freq dataset.
pred_df = pd.read_csv(home_dir+"models/dbnsfp/outputs/dbnsfp_outputs/popu_freq_preds.txt", sep="\t")
pred_df.drop(columns=['Polyphen2_HDIV_score'], inplace=True)
pred_df = pred_df.loc[pred_df[["#chr", "pos(1-based)", "ref", "alt"]].drop_duplicates(keep="first").index] # for a single chromosomal position, a model can have multiple outputs from dbnsfp, so removing them

pred_df.rename(columns={"#chr":"chrom", "pos(1-based)":"chrom_pos", "ref":"ref_allele", "alt":"alt_allele"}, inplace=True)
print(f"#-of SNVs found from dbNSFP: {pred_df.shape[0]}")

variants_df["chrom"] = variants_df["chrom_acc_version"].apply(lambda x: int(x[x.index("_")+1:x.index(".")])) # taking only chromosom number for dbNSFP inputs

print("Log: merging dbNSFP prediction scores with ground truth population freq data ...")
result_df = variants_df.merge(pred_df, how="inner", on=["chrom", "chrom_pos", "ref_allele", "alt_allele"])#, right_on=["#chr", "pos(1-based)", "ref", "alt"])
result_df = result_df.drop_duplicates(keep="first")
print(result_df.columns)
print(result_df.shape)
result_df["class"].value_counts()

/tmp/ipykernel_2573965/2035382353.py:2: DtypeWarning: Columns (13,14,15,16) have mixed types. Specify dtype option on import or set low_memory=False.
  pred_df = pd.read_csv(home_dir+"models/dbnsfp/outputs/dbnsfp_outputs/popu_freq_preds.txt", sep="\t")


#-of SNVs found from dbNSFP: 2367960
Log: merging dbNSFP prediction scores with ground truth population freq data ...
Index(['snp_id', 'chrom_acc_version', 'chrom_pos', 'ref_allele', 'alt_allele',
       'prot_acc_version', 'prot_pos', 'wt', 'mut', 'wt_population',
       'mut_poulation', 'wt_freq', 'mt_freq', 'class', 'chrom', 'aaref',
       'aaalt', 'MetaRNN_score', 'MVP_score', 'SIFT_score',
       'Polyphen2_HVAR_score', 'CADD_raw', 'REVEL_score',
       'integrated_fitCons_score', 'phyloP17way_primate',
       'phastCons17way_primate', 'bStatistic'],
      dtype='object')
(786385, 27)


Singleton     445925
Ultra-rare    298006
Rare           26419
Common         16035
Name: class, dtype: int64

In [4]:
# computing average scores for each method for each row
def compute_avg(x):
    x = str(x).split(";")
    return np.mean([float(i) for i in x if i!="."]) 

model_names = ['MetaRNN_score', 'MVP_score', 'SIFT_score', 'Polyphen2_HVAR_score', 'CADD_raw', 'REVEL_score', 'integrated_fitCons_score', 'phyloP17way_primate', 'phastCons17way_primate', 'bStatistic']
print("Log: missing values in percentages ...")
for model_name in model_names:
    model_scores = result_df[model_name].apply(compute_avg) # can have multiple scores, ie '0.4573521;0.4573521;0.4573521;0.4573521'. taking the avg
    result_df[model_name] = model_scores
    
    missing, total = result_df[pd.isna(result_df[model_name])].shape[0], result_df.shape[0]
    print(f"\t{model_name}: ({missing}/{total})*100 = {(missing / total) * 100:.4f}")
    # break

Log: missing values in percentages ...


/projects/ashehu/akabir4/venvs/hopper_variant_effect_analysis_mine/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,


	MetaRNN_score: (2016/786385)*100 = 0.2564
	MVP_score: (23260/786385)*100 = 2.9578
	SIFT_score: (36950/786385)*100 = 4.6987
	Polyphen2_HVAR_score: (54050/786385)*100 = 6.8732
	CADD_raw: (0/786385)*100 = 0.0000
	REVEL_score: (33515/786385)*100 = 4.2619
	integrated_fitCons_score: (2790/786385)*100 = 0.3548
	phyloP17way_primate: (43/786385)*100 = 0.0055
	phastCons17way_primate: (43/786385)*100 = 0.0055
	bStatistic: (13107/786385)*100 = 1.6667


In [5]:
# computing number of supervised models score available for each row
supervised_method_cols = ['MetaRNN_score', 'MVP_score', 'SIFT_score', 'Polyphen2_HVAR_score', 'CADD_raw', 'REVEL_score']
def get_num_of_methods_have_prediction_on_this_row(row):
    n = 0
    for col in supervised_method_cols:
        # print(pd.isna(row[col]))
        if not pd.isna(row[col]):
            n += 1
    # print(n)
    return n

for i in range(result_df.shape[0]):
    if i%10000==0: print(i)
    result_df.loc[i, "n_methods_having_preds"] = get_num_of_methods_have_prediction_on_this_row(result_df.loc[i])

print(result_df.shape)
print(result_df.columns)

0
10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000
180000
190000
200000
210000
220000
230000
240000
250000
260000
270000
280000
290000
300000
310000
320000
330000
340000
350000
360000
370000
380000
390000
400000
410000
420000
430000
440000
450000
460000
470000
480000
490000
500000
510000
520000
530000
540000
550000
560000
570000
580000
590000
600000
610000
620000
630000
640000
650000
660000
670000
680000
690000
700000
710000
720000
730000
740000
750000
760000
770000
780000
(786385, 28)
Index(['snp_id', 'chrom_acc_version', 'chrom_pos', 'ref_allele', 'alt_allele',
       'prot_acc_version', 'prot_pos', 'wt', 'mut', 'wt_population',
       'mut_poulation', 'wt_freq', 'mt_freq', 'class', 'chrom', 'aaref',
       'aaalt', 'MetaRNN_score', 'MVP_score', 'SIFT_score',
       'Polyphen2_HVAR_score', 'CADD_raw', 'REVEL_score',
       'integrated_fitCons_score', 'phyloP17way_primate',
       'phastCons17way_primate', 'bStatistic', 'n_m

In [6]:
# keeping only those that for which at leat 3 out of 6 supervised models' predictions.
result_df = result_df[result_df["n_methods_having_preds"]>=3]
print(result_df.shape)
result_df["class"].value_counts()

(780183, 28)


Singleton     442824
Ultra-rare    295960
Rare           26002
Common         15397
Name: class, dtype: int64

In [7]:
# recomputing the missing values to see how much reduced
model_names = ['MetaRNN_score', 'MVP_score', 'SIFT_score', 'Polyphen2_HVAR_score', 'CADD_raw', 'REVEL_score', 'integrated_fitCons_score', 'phyloP17way_primate', 'phastCons17way_primate', 'bStatistic']
for model_name in model_names:
    missing, total = result_df[pd.isna(result_df[model_name])].shape[0], result_df.shape[0]
    print(f"\t{model_name}: ({missing}/{total})*100 = {(missing / total) * 100:.4f}")

	MetaRNN_score: (0/780183)*100 = 0.0000
	MVP_score: (17058/780183)*100 = 2.1864
	SIFT_score: (30748/780183)*100 = 3.9411
	Polyphen2_HVAR_score: (47848/780183)*100 = 6.1329
	CADD_raw: (0/780183)*100 = 0.0000
	REVEL_score: (27313/780183)*100 = 3.5008
	integrated_fitCons_score: (1427/780183)*100 = 0.1829
	phyloP17way_primate: (31/780183)*100 = 0.0040
	phastCons17way_primate: (31/780183)*100 = 0.0040
	bStatistic: (12606/780183)*100 = 1.6158


In [8]:
# apply seq len filter here
from data_loader import get_protein_sequences
protid_seq_dict = get_protein_sequences(home_dir, max_seq_len=1022, return_type="protid_seq_dict", data_type="popu_freq")


Log: Loading combined fasta iterator ...
#-protein sequences (seq-len<=1022): 15962


In [24]:
prots = [] # having at least 1 variants for each class
data = []
n_unique_prots = result_df['prot_acc_version'].unique().shape[0]
for i, prot_acc_version in enumerate(result_df['prot_acc_version'].unique()):
    x = result_df[result_df['prot_acc_version']==prot_acc_version].copy()
    class_dict = x["class"].value_counts().to_dict()
    if len(class_dict)==4 and prot_acc_version in protid_seq_dict:
        seq = protid_seq_dict[prot_acc_version]
        seq_len = len(seq)
        print(f"{i}|{n_unique_prots}", prot_acc_version, class_dict, f"seq-len: {seq_len}")
        x["seq_len"] = int(seq_len)
        x["seq"] = seq
        prots.append(prot_acc_version)
        data.append(x[x["class"]=="Common"].sample(1, random_state=29))
        data.append(x[x["class"]=="Rare"].sample(1, random_state=29))
        data.append(x[x["class"]=="Ultra-rare"].sample(1, random_state=29))
        data.append(x[x["class"]=="Singleton"].sample(1, random_state=29))
    # if i==10: break
    
print(len(prots))
data = pd.concat(data)
print(data.shape)
print(data.columns)
data

1|15129 NP_064519.2 {'Singleton': 27, 'Ultra-rare': 18, 'Rare': 1, 'Common': 1} seq-len: 564
3|15129 NP_004276.2 {'Ultra-rare': 108, 'Singleton': 83, 'Rare': 3, 'Common': 1} seq-len: 791
4|15129 NP_758872.1 {'Singleton': 38, 'Ultra-rare': 31, 'Rare': 2, 'Common': 1} seq-len: 989
6|15129 NP_065806.1 {'Singleton': 21, 'Ultra-rare': 18, 'Rare': 1, 'Common': 1} seq-len: 453
9|15129 NP_001124389.2 {'Ultra-rare': 43, 'Singleton': 36, 'Rare': 21, 'Common': 11} seq-len: 483
10|15129 NP_003357.2 {'Singleton': 30, 'Ultra-rare': 12, 'Rare': 2, 'Common': 1} seq-len: 453
11|15129 NP_064560.2 {'Singleton': 59, 'Ultra-rare': 39, 'Rare': 2, 'Common': 2} seq-len: 650
14|15129 NP_112228.1 {'Singleton': 14, 'Ultra-rare': 10, 'Rare': 5, 'Common': 3} seq-len: 167
17|15129 NP_067022.1 {'Singleton': 44, 'Ultra-rare': 21, 'Rare': 2, 'Common': 1} seq-len: 450
18|15129 NP_000693.1 {'Singleton': 39, 'Ultra-rare': 21, 'Common': 1, 'Rare': 1} seq-len: 1020
23|15129 NP_001171754.1 {'Singleton': 48, 'Ultra-rare': 38

,snp_id,chrom_acc_version,chrom_pos,ref_allele,alt_allele,prot_acc_version,prot_pos,wt,mut,wt_population,...,Polyphen2_HVAR_score,CADD_raw,REVEL_score,integrated_fitCons_score,phyloP17way_primate,phastCons17way_primate,bStatistic,n_methods_having_preds,seq_len,seq
107,rs41288789,NC_000002.12,26944749,G,A,NP_064519.2,512,A,T,218102,...,0.029,2.637780,0.280,0.648238,0.676,0.972,558.0,5.0,564,MLANSASVRILIKGGKVVNDDCTHEADVYIENGIIQQVGRELMIPG...
98,rs151073506,NC_000002.12,26934674,C,T,NP_064519.2,296,T,M,203738,...,0.841,2.417774,0.344,0.706298,0.549,0.861,632.0,6.0,564,MLANSASVRILIKGGKVVNDDCTHEADVYIENGIIQQVGRELMIPG...
103,rs144366814,NC_000002.12,26927400,G,A,NP_064519.2,190,V,I,203740,...,0.303,3.081825,0.463,0.626454,0.676,0.989,639.0,6.0,564,MLANSASVRILIKGGKVVNDDCTHEADVYIENGIIQQVGRELMIPG...
77,rs777187108,NC_000002.12,26924926,G,A,NP_064519.2,101,V,I,35424,...,0.660,3.229382,0.534,0.643511,0.618,0.209,636.0,6.0,564,MLANSASVRILIKGGKVVNDDCTHEADVYIENGIIQQVGRELMIPG...
382,rs34603401,NC_000001.11,9245386,A,C,NP_004276.2,151,D,A,225918,...,0.338,3.875482,0.561,0.706548,0.691,0.748,928.0,5.0,791,MWNMLIVAMCLALLGCLQAQELQGHVSIILLGATGDLAKKYLWQGL...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
785630,rs766123479,NC_000010.11,96351409,G,A,NP_149984.1,14,T,M,23036,...,0.998,4.062150,0.251,0.487112,0.676,0.196,451.0,6.0,141,MSFSLNFTLPANTTSSPVTGGKETDCGPSLGLAAGIPLLVATALLV...
786031,rs3745746,NC_000019.10,48034328,A,G,NP_062829.1,128,V,A,339566,...,0.001,-0.687187,0.075,0.487112,-0.665,0.241,583.0,5.0,173,MQFPMGPACIFLRKGIAEKQRERPLGQDEIEELREAFLEFDKDRDG...
786027,rs111484042,NC_000019.10,48040712,C,T,NP_062829.1,44,R,Q,65846,...,0.005,1.906385,0.058,0.500041,0.589,0.994,579.0,5.0,173,MQFPMGPACIFLRKGIAEKQRERPLGQDEIEELREAFLEFDKDRDG...
786029,rs10425606,NC_000019.10,48034271,A,C,NP_062829.1,147,I,S,162822,...,0.563,4.135370,0.408,0.487112,0.756,0.879,583.0,6.0,173,MQFPMGPACIFLRKGIAEKQRERPLGQDEIEELREAFLEFDKDRDG...


In [2]:
result_df = data.copy()
out_filepath = home_dir+f"models/aa_common/datasets_population_freq/SNVs_with_popu_freq_sampled"
result_df.to_csv(out_filepath+".tsv", sep="\t")
# result_df = pd.read_csv(out_filepath+".tsv", sep="\t")
print("\nLog: Creating merged fasta document ...")
protein_acc_list = list(result_df["prot_acc_version"].unique())
print(len(protein_acc_list))
from utils.ncbi_proteins import create_combined_fasta
create_combined_fasta(protein_acc_list, out_filepath+".fasta", home_dir)


Log: Creating merged fasta document ...
5725
0 NP_064519.2 Already existis
1 NP_004276.2 Already existis
2 NP_758872.1 Already existis
3 NP_065806.1 Already existis
4 NP_001124389.2 Already existis
5 NP_003357.2 Already existis
6 NP_064560.2 Already existis
7 NP_112228.1 Already existis
8 NP_067022.1 Already existis
9 NP_000693.1 Already existis
10 NP_001171754.1 Already existis
11 NP_079203.4 Already existis
12 NP_001164225.1 Already existis
13 NP_079539.2 Already existis
14 NP_115827.1 Already existis
15 NP_001157912.1 Already existis
16 NP_002617.3 Already existis
17 NP_001372577.1 Already existis
18 NP_112599.2 Already existis
19 NP_001071174.1 Already existis
20 NP_001035940.1 Already existis
21 NP_073753.3 Already existis
22 NP_000180.2 Already existis
23 NP_055106.1 Already existis
24 NP_788276.1 Already existis
25 NP_061732.1 Already existis
26 NP_001005356.1 Already existis
27 NP_060988.3 Already existis
28 NP_057118.1 Already existis
29 NP_001167594.1 Already existis
30 NP_0

In [25]:
# recomputing the missing values to see how much reduced

model_names = ['MetaRNN_score', 'MVP_score', 'SIFT_score', 'Polyphen2_HVAR_score', 'CADD_raw', 'REVEL_score', 'integrated_fitCons_score', 'phyloP17way_primate', 'phastCons17way_primate', 'bStatistic']
for model_name in model_names:
    missing, total = result_df[pd.isna(result_df[model_name])].shape[0], result_df.shape[0]
    print(f"\t{model_name}: ({missing}/{total})*100 = {(missing / total) * 100:.4f}")

	MetaRNN_score: (0/22900)*100 = 0.0000
	MVP_score: (5085/22900)*100 = 22.2052
	SIFT_score: (1034/22900)*100 = 4.5153
	Polyphen2_HVAR_score: (1829/22900)*100 = 7.9869
	CADD_raw: (0/22900)*100 = 0.0000
	REVEL_score: (983/22900)*100 = 4.2926
	integrated_fitCons_score: (70/22900)*100 = 0.3057
	phyloP17way_primate: (3/22900)*100 = 0.0131
	phastCons17way_primate: (3/22900)*100 = 0.0131
	bStatistic: (438/22900)*100 = 1.9127
